In [3]:
# !pip install torch==1.10
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.html
# !pip install torch-geometric
# !pip install ipykernel
# !pip install wandb

In [4]:
import wandb

wandb.init(project="relational-autoencoder", entity="link-prediction-gnn", )
wandb.config = {
  "learning_rate": 0.005,
  "epochs": 25,
}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: di3go (use `wandb login --relogin` to force relogin)
/Users/di3go/Desktop/neural-graph/venv/lib/python3.9/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [5]:
import torch, torch_geometric

In [6]:
DS_PATH= './FB15k-237'
DS_NAME= 'FB15k-237'
ds = torch_geometric.datasets.RelLinkPredDataset(DS_PATH, DS_NAME)

edge_indices_train = ds.data.edge_stores[0]['train_edge_index']
edge_types_train = ds.data.edge_stores[0]['train_edge_type']

edges_test = ds.data.edge_stores[0]['test_edge_index'].detach()
edges_type_test = ds.data.edge_stores[0]['test_edge_type'].detach()

In [7]:
N_RELATIONS = 237
N_NODES = ds.data.num_nodes
NODE_EMBEDDING_SIZE = 16

rgcn = torch_geometric.nn.RGCNConv(N_NODES, NODE_EMBEDDING_SIZE, N_RELATIONS)
r_matrices = torch.randn(N_RELATIONS, NODE_EMBEDDING_SIZE, NODE_EMBEDDING_SIZE)


In [8]:
import matplotlib.pyplot as plt

# Create binary dense matrix for relation type == 1

train_links_truth = torch.sparse_coo_tensor(edge_indices_train, edge_types_train, (N_NODES,N_NODES) ).to_dense().float()
# test_links_truth = torch.sparse_coo_tensor(Xi_test[:,test_f], Xr_test[test_f], (N_NODES,N_NODES)).to_dense().float()

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(rgcn.parameters(), lr=wandb.config['learning_rate'])

def score_reconstruction(edge_scores, edge_tags, edge_tags_truth):
    loss = criterion(edge_scores, edge_tags_truth)
    acc = ( 1 - (edge_tags - edge_tags_truth).abs()).sum() / (N_NODES**2)
    return loss, acc

for epoch in range(wandb.config['epochs']):
    perm = torch.randperm(edge_indices_train.size(0))
    idx = perm[:edge_indices_train.size(0) * 7 // 10]
    edge_indices_sample = edge_indices_train[:,idx]
    edge_types_sample = edge_types_train[idx]
    nodes_embeddings = rgcn(None, edge_indices_sample, edge_types_sample)
    optimizer.zero_grad()
    # edges_scores = torch.zeros((N_RELATIONS, N_NODES, N_NODES))
    tot_loss = torch.zeros(1)
    avg_acc, avg_exp = 0, 0
    for r_id, r_mat in enumerate(r_matrices):

        # print(f'relation {r_id}')
        edge_scores = nodes_embeddings @ r_mat @ nodes_embeddings.T
        # print('scored', edge_scores)
        edge_tags = edge_scores.sigmoid().round()
        # print('tagged', edge_tags)

        r_indices = edge_indices_train  [:, edge_types_train == r_id]
        edge_tags_truth = torch.zeros((N_NODES, N_NODES))
        edge_tags_truth[r_indices] = 1
        expansion = edge_tags.sum()/ edge_tags_truth.sum()
        loss, acc = score_reconstruction(edge_scores, edge_tags, edge_tags_truth)
        
        tot_loss += loss
        avg_acc += (avg_acc * (r_id - 1) + acc ) / r_id
        avg_exp += (avg_exp * (r_id - 1) + expansion ) / r_id
        print(f"Epoch: {epoch} | RID: {r_id} | Acc: {acc} | Exp rate : {expansion} ")

        if r_id > 3:
            break

    tot_loss.backward()
    optimizer.step()
    
    wandb.log({"loss": tot_loss})
    wandb.log({"accuracy": avg_acc})
    wandb.log({"expansion": avg_exp})


Epoch: 0 | RID: 0 | Acc: 0.500076413154602 | Exp rate : 8.716093063354492 
Epoch: 0 | RID: 1 | Acc: 0.5000501871109009 | Exp rate : 14.657217025756836 
Epoch: 0 | RID: 2 | Acc: 0.499980092048645 | Exp rate : 1.7024117708206177 
Epoch: 0 | RID: 3 | Acc: 0.49998733401298523 | Exp rate : 33.50543975830078 
Epoch: 0 | RID: 4 | Acc: 0.5000250935554504 | Exp rate : 3.664473295211792 
Epoch: 1 | RID: 0 | Acc: 0.4635230600833893 | Exp rate : 9.54342269897461 
Epoch: 1 | RID: 1 | Acc: 0.6574075222015381 | Exp rate : 9.959146499633789 
Epoch: 1 | RID: 2 | Acc: 0.46944522857666016 | Exp rate : 2.0579135417938232 
Epoch: 1 | RID: 3 | Acc: 0.6832994818687439 | Exp rate : 20.9314022064209 
Epoch: 1 | RID: 4 | Acc: 0.5766010284423828 | Exp rate : 2.9424221515655518 
Epoch: 2 | RID: 0 | Acc: 0.8296726942062378 | Exp rate : 2.5666964054107666 
Epoch: 2 | RID: 1 | Acc: 0.8780379295349121 | Exp rate : 3.0951712131500244 
Epoch: 2 | RID: 2 | Acc: 0.4350912868976593 | Exp rate : 2.6464414596557617 
Epoch: 

wandb: Network error (ReadTimeout), entering retry loop.


Epoch: 10 | RID: 3 | Acc: 0.992411196231842 | Exp rate : 0.4914839565753937 
Epoch: 10 | RID: 4 | Acc: 0.8635582327842712 | Exp rate : 1.386513304169057e-07 
Epoch: 11 | RID: 0 | Acc: 0.9426448345184326 | Exp rate : 0.0 
Epoch: 11 | RID: 1 | Acc: 0.9658894538879395 | Exp rate : 0.0 
Epoch: 11 | RID: 2 | Acc: 0.5227153897285461 | Exp rate : 2.624958038330078 
Epoch: 11 | RID: 3 | Acc: 0.993071973323822 | Exp rate : 0.5357611179351807 
Epoch: 11 | RID: 4 | Acc: 0.8635582327842712 | Exp rate : 6.932566520845285e-08 
Epoch: 12 | RID: 0 | Acc: 0.9426448345184326 | Exp rate : 0.0 
Epoch: 12 | RID: 1 | Acc: 0.9658894538879395 | Exp rate : 0.0 
Epoch: 12 | RID: 2 | Acc: 0.599791407585144 | Exp rate : 2.3625457286834717 
Epoch: 12 | RID: 3 | Acc: 0.9935746192932129 | Exp rate : 0.5694421529769897 
Epoch: 12 | RID: 4 | Acc: 0.8635582327842712 | Exp rate : 0.0 
Epoch: 13 | RID: 0 | Acc: 0.9426448345184326 | Exp rate : 0.0 
Epoch: 13 | RID: 1 | Acc: 0.9658894538879395 | Exp rate : 0.0 
Epoch: 13 |